In [ ]:
import numpy as np
import pandas as pd

import os
import re


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification,TrainingArguments,Trainer, AutoTokenizer
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:

train_data = pd.read_csv("train_data.csv", sep='|')
validate_data = pd.read_csv("validate_data.csv", sep='|')
test_data = pd.read_csv("test_data.csv", sep='|')

display(train_data)
display(validate_data)
display(test_data)

In [ ]:

train_data = train_data.reset_index(drop=True)
validate_data = validate_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [ ]:
print("Training size: {}".format(len(train_data)))
print("Validation size: {}".format(len(validate_data)))
print("Testing size: {}".format(len(test_data)))

In [ ]:
#Escolhendo a GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", from_tf=True)
model.config.num_labels = 2

In [ ]:
#Desabilitando a atualização dos gradientes para acelerar o processamento e diminuir o consumo
for param in model.parameters():
    param.requires_grad = False
# Comentar esta seção para ativar o ajuste de gradientes

In [ ]:
# Definindo a arquitetura da camada classificadora
# composta por várias camadas densas e de ativação para processamento não linear e uma camada Softmax 
# para obter as probabilidades de cada classe.
model.classifier = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Linear(64, 2),
    nn.Softmax(dim=1)
)

In [ ]:
model = model.to(device)

# Alocando o modelo e o critério de perda na GPU
criterion = nn.MSELoss().to(device)


optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)

In [ ]:
def clean_text(text):
  #limpando String para caso a fonte de dados não seja pre-processada
  return text.replace('\n', ' ').replace('\t', ' ').replace('   ', ' ').replace('  ', ' ')
 

def preprocess_text(text):
    text = clean_text(text)
    parts = []

    text_len = len(text.split(' '))
    delta = 300 
    max_parts = 5 
    nb_cuts = int(text_len / delta)
    nb_cuts = min(nb_cuts, max_parts)
    
    
    for i in range(nb_cuts + 1):
        text_part = ' '.join(text.split(' ')[i * delta: (i + 1) * delta])
        parts.append(tokenizer.encode(text_part, return_tensors="pt", max_length=500).to(device))

    return parts

def testText(text):
    text_parts = preprocess_text(text)
    overall_output = torch.zeros((1,2)).to(device)
    try:
        for part in text_parts:
            if len(part) > 0:
                overall_output += model(part.reshape(1, -1))[0]
    except RuntimeError:
        print("out of memory")

    overall_output = F.softmax(overall_output[0], dim=-1)

    value, result = overall_output.max(0)

    term = "falso"
    if result.item() == 0:
        term = "verdadeiro"

    print("{} : {}%".format(term, value.item() * 100))
    return term

In [ ]:
class BertFakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_seq_length=512):
        self._labels = torch.tensor(df["label"].values, dtype=torch.long)
        self._encodings = {"input_ids": [],
                           "token_type_ids": [],
                           "attention_mask": []}

        for txt in df["text"].values:
            enc_dict = tokenizer.encode_plus(
                text=txt,
                add_special_tokens=True,
                max_length=max_seq_length,
                return_token_type_ids=True,
                padding="max_length",
                return_attention_mask=True,
                return_tensors="pt",
                truncation=True,
            )
            
            if len(enc_dict["input_ids"][0]) > max_seq_length:
                print("Truncamento detectado após adição de tokens especiais!")

            for k, v in enc_dict.items():
                self._encodings[k].append(v[0])
        
    def __getitem__(self, idx):
        item = {key: value[idx] for key, value in self._encodings.items()}
        item["labels"] = self._labels[idx]
        return item

    def __len__(self):
        return len(self._labels)

In [ ]:
train_dataTK = BertFakeNewsDataset(train_data,tokenizer)
validate_dataTK = BertFakeNewsDataset(validate_data,tokenizer)
test_dataTK = BertFakeNewsDataset(test_data,tokenizer)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1_score": f1_score(labels, predictions, average='weighted'),
    }

In [ ]:
train_args = TrainingArguments(
    output_dir='C:\TCC',
    evaluation_strategy="epoch",
    learning_rate=2.5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_strategy="epoch",
    save_steps=1000,
    save_total_limit=10,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataTK,
    eval_dataset=validate_dataTK,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(train_dataTK)

In [ ]:
trainer.evaluate(validate_dataTK)

In [ ]:
trainer.evaluate(test_dataTK) 

In [ ]:
model = trainer.model

In [ ]:
falso = """Mais uma pérola da "senhora que estoca vento". Em nota, ela sugere que a PF "tortura" investigados.  A ex-presidente impeachmada Dilma Rousseff continua falando pelos cotovelos.  (observação: Em seu texto, ela não usou a palavra "tortura" [...] mas para quem sabe ler, um pingo é letra!) Durante a última semana, Dilma teve a audácia (a petulância, a ousadia, destemor) de dizer que os investigadores da Polícia Federal estão ameaçando os delatores da Lava-Jato. Dilma também atacou o ministro relator da Lava-Jato, Edson Fachin, e o acusou de abrir as delações de João Santana e Mônica Moura para a imprensa antes sem antes dar acesso dos documentos a seus advogados. Resumindo, Dilma é honesta [...] já Edson Fachin (Ministro do STF) e a Polícia Federal são os "bandidos da história" Em uma nota divulgada na última sexta-feira, a "senhora que estoca vento" também disse que todos os delatores são mentirosos. abaixo a íntegra da nota: 1. Infelizmente, chega tarde a decisão do relator da Lava Jato no Supremo Tribunal Federal, ministro Edson Fachin, suspendendo o sigilo dos depoimentos de João Santana e Monica Moura. 2. Há semanas, a defesa da presidenta eleita Dilma Rousseff havia feito tal pedido ao Tribunal Superior Eleitoral, a fim de apresentar suas alegações finais ao relator do caso das contas de campanha, ministro Herman Benjamin. 3. A defesa foi prejudicada pela negativa do relator. Não foi possível cotejar os depoimentos prestados pelo casal à Justiça Eleitoral e na Lava Jato. 4. As contradições e falsos testemunhos foram vislumbrados, apesar disso, pelo que foi divulgado amplamente pela imprensa, na velha estratégia do vazamento seletivo dos depoimentos – uma rotina nos últimos tempos. 5. Agora mesmo, os depoimentos são entregues à imprensa, mas não repassados oficialmente à defesa da presidente eleita. 6. Dilma Rousseff, contudo, reitera o que apontou antes: João Santana e Monica Moura prestaram falso testemunho e faltaram com a verdade em seus depoimentos, provavelmente pressionados pelas ameaças dos investigadores. 7. Apesar de tudo, a presidente eleita acredita na Justiça e sabe que a verdade virá à tona e será restabelecida."""

Verdadeiro = """Saúde de Lula é 'excelente', atestam exames feitos hoje

O ex-presidente Luiz Inácio Lula da Silva encontra-se em "excelente condição de saúde e sem qualquer evidência de neoplasia", diz o boletim assinado pelos médicos Antonio Carlos Onofre de Lira, diretor técnico médico, e Paulo Cesar Ayroza Galvão, diretor clínico do hospital sírio-libanês.

Lula foi submetido a exames clínicos, laboratoriais, PET-CT, ressonância nuclear magnética e laringoscopia durante toda a manhã deste sábado, 1. Todos de rotina. Essa rotina vem sendo repetida desde 2011, quando foi detectado um câncer na laringe do ex-presidente, que se submeteu a tratamento e deve fazer o controle por cinco anos. O último exame havia sido realizado em 10 de agosto. As equipes que acompanham o ex-presidente são coordenadas pelos médicos Roberto Kalil Filho e Artur Katz. Lula deixou o hospital sem falar com a imprensa.
"""

testText(falso)
testText(Verdadeiro)

In [ ]:
torch.save(model, 'modelo.pth')

In [ ]:
from huggingface_hub import notebook_login
model.push_to_hub("",use_auth_token='')

tokenizer.push_to_hub("",use_auth_token='')